get a small data set to work on the bigram language model with

it's a book with the licenceing pages and intro pages removed to not affect the prediction

**Downloading the Text File:**

In [12]:
!wget https://github.com/Infatoshi/fcc-intro-to-llms/blob/main/wizard_of_oz.txt

--2023-11-28 08:32:48--  https://github.com/Infatoshi/fcc-intro-to-llms/blob/main/wizard_of_oz.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277641 (271K) [text/plain]
Saving to: ‘wizard_of_oz.txt.1’

wizard_of_oz.txt.1  100%[===================>] 271.13K  1.10MB/s    in 0.2s    

2023-11-28 08:32:48 (1.10 MB/s) - ‘wizard_of_oz.txt.1’ saved [277641/277641]



**Importing Libraries:**

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device ='cuda' if torch.cuda.is_available() else 'cpu'

**Reading and Preprocessing the Text File:**

In [14]:
with open('wizard_of_oz.txt','r',encoding='utf-8')as f :
  text=f.read()

chars=sorted(set(text))
print(chars)

[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', '\ufeff']


In [15]:
vocabulary_size=len(chars)

**Mapping Characters to Integers:**


Creating mappings between characters and integers and vice versa. Also, defining functions to encode a string into a list of integers and decode a list of integers back to a string.


In [16]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

**Creating Tensor from Encoded Text:**

Converting the encoded text into a PyTorch tensor of type long.
transforming our text corpus into a tensor data

In [17]:
data = torch.tensor(encode(text),dtype=torch.long)
data

tensor([83,  2, 72,  ..., 75,  2, 84])

making tragets out of data for predictions using blocks method

In [18]:
block_size=8
batch_size=4

**Defining Training and Validation Data:**



In [19]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]



tensor([  2376,  40924, 117207, 134059])
inputs :
 tensor([[70, 75,  0, 65, 70,  0, 74, 61],
        [26, 54,  2,  0, 79, 64, 65, 75],
        [76, 71, 68, 60,  0, 77, 75,  0],
        [11,  0, 46, 64, 61, 70,  0, 64]], device='cuda:0') 
 targets :
 tensor([[75,  0, 65, 70,  0, 74, 61],
        [54,  2,  0, 79, 64, 65, 75],
        [71, 68, 60,  0, 77, 75,  0],
        [ 0, 46, 64, 61, 70,  0, 64]], device='cuda:0')


**Batch Creation Function:**


This function generates a batch of input and target sequences for training or validation.



In [ ]:

def get_batch(split):
  data=train_data if split=='train' else val_data
  ix =torch.randint(len(data)-block_size,(batch_size,))
  print(ix)
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size] for i in ix])
  x,y=x.to(device),y.to(device)
  return x,y

x,y = get_batch('train')
print('inputs :\n',x,'\n','targets :\n',y)

**Print Example Training Instances:**

Printing example training instances with increasing context and corresponding targets.



In [20]:

x=train_data[:block_size]
y=train_data[1:block_size+1] #offesting by one

for i in range(block_size):
  context=x[:i+1]
  target=y[i]
  print('when input is ',context, ' the target is ',target)

when input is  tensor([83])  the target is  tensor(2)
when input is  tensor([83,  2])  the target is  tensor(72)
when input is  tensor([83,  2, 72])  the target is  tensor(57)
when input is  tensor([83,  2, 72, 57])  the target is  tensor(81)
when input is  tensor([83,  2, 72, 57, 81])  the target is  tensor(68)
when input is  tensor([83,  2, 72, 57, 81, 68])  the target is  tensor(71)
when input is  tensor([83,  2, 72, 57, 81, 68, 71])  the target is  tensor(57)
when input is  tensor([83,  2, 72, 57, 81, 68, 71, 57])  the target is  tensor(60)


**Bigram Language Model Class:**

The forward method takes an input index and optional targets. It calculates logits (unnormalized scores) using the token embedding table. If targets are provided, it calculates the cross-entropy loss between predicted logits and actual targets.


The generate method generates new sequences given an initial context (index) and the maximum number of new tokens to generate. It repeatedly predicts the next token, samples from the distribution, and appends it to the sequence.

Creating an instance of the BigramLanguageModel, moving it to the specified device, and initializing a context tensor for sequence generation.

In summary, this code defines a simple bigram language model using PyTorch. It trains the model on a text dataset and showcases how to generate new sequences based on the learned patterns. The generated sequences reflect the model's understanding of the language structure based on the training data.

In [27]:
class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,index,targets=None):
    logits=self.token_embedding_table(index)

    if targets is None:
      loss=None
    else:
      B,T,C= logits.shape     #batch time channel
      logits=logits.view(B*T,C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,index,max_new_tokens):
    #index is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predictions
      logits,loss= self.forward(index)
      #focus only o the last time step
      logits=logits[:,-1,:] #becomes (B,C)
      #apply softmax to get probabilities
      probs=F.softmax(logits,dim=-1)
      #sample from distribution
      index_next=torch.multinomial(probs,num_samples=1) #(B,1)
      #append sampled index to runing sequence
      index=torch.cat((index,index_next),dim=1)#(B,T+1)
    return index

model =BigramLanguageModel(vocabulary_size)
m=model.to(device)
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


 Yg0-﻿4v;{\)IWE3)Kwj,bcc;﻿,*I{[]K&[s_YGBQA﻿ZU·i8*O_OXH%Ii﻿2m·EXMtI.A(eo%lPY9c2i[\.sxfy4{mO'\\*8h5,S=;Q0iRcSAys'dPU}('Gj)qWKm) [:6ye·!UFyAv:Ue"Y[7/WwuUSbzikJJN7lwCZP"(\gRxW·d W7dP3pHUynOL?!ql/X'n﻿hc1%,]F-nV1X,}p[73"!0w3%PTq[YCcctVV1Vhi·)i·dP4ZKV,2*i*rWsxftNloCrWVb*"A!v\CXg_KMja8zX8"RjLW8MXP8SI7SKJu43gDJRv_(\bd]n%d'GBJz{IH,o3gXL;q·L:m!X8d8M:8﻿%*fd/ZkELN&J8QU!G1M:Ao!mk";zE.VmP;1m&·?}F[v Y?SLM-84QWtj01"JV1 PKeOLNl[8VbO{)M- 8p·i·QA3T/,mWW:SP4. El4,y%4Vr7;-a)("&,"W-:9.0KcIgaJi5_pXi·CC=·(7NFjv8ZUF[7[9.]
